# Script demonstrating comparisons between the FEL and a given EL for FPM
In FPM the FEL with contain a target data model which the EL structure of a given company can be compared against.
In this script the datatypes are compared to demonstrate the possibility to use a graph database for EL preprocessing in constructing a FPM

In [1]:
# Import libraries
import json
import pandas as pd
import numpy
import re

In [2]:
#define the Federated Event Log (FEL) and a given source EL
object_event_log_name = 'BPI_C_2019_FEL_synthetic_' #Here is the FEL Meta-DataFrame which has to be created through script 2
subject_event_log_name = 'BPI_C_2019_3_final_synthetic'

In [3]:
#Optionally Clean up previous relationships containing CHANGE or FIT in case they where generated falsely
print('// Delete fit and change relationships.')
print(f'MATCH (object_node:{object_event_log_name}) - [rel:CHANGE|FIT] -> (subject_node:{subject_event_log_name}) DELETE rel;')
print()

// Delete fit and change relationships.
MATCH (object_node:BPI_C_2019_FEL_synthetic_) - [rel:CHANGE|FIT] -> (subject_node:BPI_C_2019_3_final_synthetic) DELETE rel;



In [4]:
fit_and_change_commands = ''
fit_and_change_commands += '// Delete fit and change relationships.\n'
fit_and_change_commands += f'// MATCH (object_node:{object_event_log_name}) - [rel:CHANGE|FIT] -> (subject_node:{subject_event_log_name}) DELETE rel;\n\n'

fit_and_change_commands += '// Create fit and change relationships.\n'

object_log_columns_table = pd.read_csv(f'analysis/comparison/overview_for_Comparison_{object_event_log_name}.csv')
subject_log_columns_table = pd.read_csv(f'analysis/comparison/overview_for_Comparison_{subject_event_log_name}.csv')
for relationship_name in object_log_columns_table.index:

  # of type <class 'pandas.core.series.Series'>
  object_row = object_log_columns_table.loc[relationship_name]
  subject_row = subject_log_columns_table.loc[relationship_name]

  # the Property is a property derived from the event log; this property is a column in the event log; hence the property is also called a log column
  object_log_column = object_row['Property']
  subject_log_column = subject_row['Property']

  make_FIT_relationship_command = 'MATCH '
  make_FIT_relationship_command += f'(object_node:{object_event_log_name}:`{object_log_column}`), '
  make_FIT_relationship_command += f'(subject_node:{subject_event_log_name}:`{subject_log_column}`) '
  make_FIT_relationship_command += 'WHERE object_node.`Data type` = subject_node.`Data type` '
  make_FIT_relationship_command += 'CREATE (object_node) - [:FIT] -> (subject_node);\n'

  make_CHANGE_relationship_command = 'MATCH '
  make_CHANGE_relationship_command += f'(object_node:{object_event_log_name}:`{object_log_column}`), '
  make_CHANGE_relationship_command += f'(subject_node:{subject_event_log_name}:`{subject_log_column}`) '
  make_CHANGE_relationship_command += 'WHERE object_node.`Data type` <> subject_node.`Data type` '
  make_CHANGE_relationship_command += 'CREATE (object_node) - [:CHANGE] -> (subject_node);\n'

  fit_and_change_commands += make_FIT_relationship_command
  fit_and_change_commands += make_CHANGE_relationship_command

fit_and_change_commands += '\n'

fit_and_change_commands += '// Show all nodes from the two graphs, between which we have created fit and change relationships.\n'
fit_and_change_commands += f'// MATCH (node) WHERE node:{object_event_log_name} OR node:{subject_event_log_name} RETURN (node);\n'
fit_and_change_commands += '\n'

fit_and_change_commands += '// Show any node in the graph database.\n'
fit_and_change_commands += '// MATCH (node) RETURN (node);\n'

with open(f'analysis/FIT_and_CHANGE_REL_Dataset_3.cypher', 'w') as overview_file:
  print(fit_and_change_commands, file=overview_file)

# End of Script 3 demonstrating comparisons between the FEL and a given EL for FPM
Author: Kyle Smith <br>
Script: For Masterthesis <br>
University: University of Camerino & University of Applied Sciences Northwestern Switzerland